In [3]:
import pandas as pd

# -----------------------------
# 0. 파일 경로 설정
# -----------------------------
cohort_path = "./../cohort/cohort_ver50_only_subject_id.csv"      # cohort 50 (subject_id, hadm_id 포함)
mm_path     = "./../../../data/mimic-iv-ecg/machine_measurements.csv"              # 전체 machine_measurements
out_path    = "./../cohort/ecg_ver50.csv"  # 결과 저장 경로

# -----------------------------
# 1. 데이터 로딩
# -----------------------------
cohort = pd.read_csv(cohort_path)
mm = pd.read_csv(mm_path, low_memory=False)

# -----------------------------
# 2. 공통 키(subject_id, hadm_id) 확인
#    - 둘 다 있으면 subject_id+hadm_id 기준
#    - hadm_id가 한쪽에 없으면 subject_id만 사용
# -----------------------------
keys = [c for c in ["subject_id", "hadm_id"] if c in cohort.columns and c in mm.columns]

if not keys:
    raise ValueError("cohort 또는 machine_measurements에 subject_id / hadm_id 공통 컬럼이 없습니다.")

# cohort에서 중복 제거
cohort_keys = cohort[keys].drop_duplicates()

# -----------------------------
# 3. inner join으로 겹치는 부분만 추출
# -----------------------------
filtered = mm.merge(cohort_keys, on=keys, how="inner")

print(f"[INFO] 공통 키 {keys} 기준 필터링 완료")
print(f"[INFO] 원본 machine_measurements: {len(mm):,} rows")
print(f"[INFO] 필터링 후: {len(filtered):,} rows")

# -----------------------------
# 4. 결과 저장
# -----------------------------
filtered.to_csv(out_path, index=False)
print(f"[SAVE] 저장 경로: {out_path}")


[INFO] 공통 키 ['subject_id'] 기준 필터링 완료
[INFO] 원본 machine_measurements: 800,035 rows
[INFO] 필터링 후: 27,481 rows
[SAVE] 저장 경로: ./../cohort/ecg_ver50.csv
